In [ ]:
import pandas as pd
import numpy as np
import statistics as stat
from lxml import html
import requests
from bs4 import BeautifulSoup
import csv
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')


%matplotlib inline

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    text_nt = re.sub(clean, '', text)
    return text_nt

In [ ]:
def sentiment_comment_text(url):
    """Takes Lobste'rs URL and returns sentiment score and text of post comments"""
    page = requests.get(url, timeout=5)
    #brew the soup
    soup = BeautifulSoup(page.text)
    #find all comments on Lobste.rs post
    comment_text = str(soup.find_all('p'))
    #get rid of html tags
    comment_text = remove_html_tags(comment_text)
    sia = SentimentIntensityAnalyzer()
    #run the sentiment analysis
    sentiment_score = sia.polarity_scores(comment_text)['compound']
    return pd.Series((comment_text, sentiment_score ))

In [ ]:
df[['comment_text','comment_sentiment']] = df['comments_url'].apply(sentiment_comment_text)

In [ ]:
df = pd.read_csv('C:/Users/sam_a/Downloads/uf-data-analytics-project/lobsters_full_2017_cleaned.csv', index_col='Unnamed: 0')

In [ ]:
#add new engagement score column
df["engagement_score"] = df["upvotes"] + df["downvotes"] + df["comment_count"]

In [ ]:
df.head()

In [ ]:
#save original df
og_df = df

In [ ]:
df['engagement_score'].hist()

In [ ]:
#Chart engagement score by usn
total_engagement_score_by_usn = df.groupby('username')['engagement_score'].sum()
print(total_engagement_score_by_usn)
total_engagement_score_by_usn.plot()

In [ ]:
#chart number of posts by usn
total_posts_by_usn = df.groupby('username')['engagement_score'].count()
df= total_posts_by_usn
total_posts_by_usn = df.drop(df[df < 5].index)
print(total_posts_by_usn)
total_posts_by_usn.plot()

In [ ]:
#Find average post engagement per username
avg_engagement_by_user = total_engagement_score_by_usn / total_posts_by_usn
avg_engagement_by_user = avg_engagement_by_user.dropna()
print(avg_engagement_by_user)

In [ ]:
#find the standard deviation of the engagement score
stdev_avg_engagement = stat.stdev(avg_engagement_by_user)
print(stdev_avg_engagement)

In [ ]:
#mean of th engagement score
mean_avg_engagement = stat.mean(avg_engagement_by_user)
print(mean_avg_engagement)

In [ ]:
#find standard deviation levels
one_eng_stdev = stdev_avg_engagement + mean_avg_engagement
two_eng_stdev = (stdev_avg_engagement*2) + mean_avg_engagement
three_eng_stdev = (stdev_avg_engagement*3) + mean_avg_engagement

In [ ]:
df= avg_engagement_by_user

In [ ]:
stdev1df = df.drop(df[df < one_eng_stdev].index)
stdev2df = df.drop(df[df < two_eng_stdev].index)
stdev3df = df.drop(df[df < three_eng_stdev].index)

In [ ]:
print(stdev1df)
stdev1df.plot(kind= 'bar')

In [ ]:
print(stdev2df)
stdev1df.plot(kind= 'bar')

In [ ]:
print(stdev3df)
stdev1df.plot(kind= 'bar')

In [ ]:
df = og_df

In [ ]:
df.head()

In [ ]:
#df = df.drop(df[df.comment_count < 1].index)
#comment_page = df['comments_url'].values
#comment_page= comment_page.tolist()

In [ ]:
#temp for working past bottleneck
#comment_page = 'https://lobste.rs/s/002cgs/resilience_ruby'

In [ ]:
#attemped for loop
'''#read webpage 
for URL in comment_page:
    try:
        page = requests.get(row[2], timeout=5)
    except (requests.ConnectionError, requests.HTTPError, requests.Timeout) as e:
        logging.exception(e)
    except requests.RequestException as e:
        logging.exception(e) 
        sys.exit(1)  '''

In [ ]:
sentiment_comment_text(comment_page)

In [ ]:
og_df.head()

In [ ]:
#og_df['engagement_score'].corr(og_df['comment_sentiment'])